# Imports

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
# import seaborn as sns
import warnings

import joblib

import psycopg2 as ps

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, OrdinalEncoder
from sklearn.pipeline import Pipeline

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, confusion_matrix, ConfusionMatrixDisplay

import os

In [2]:
PATH_IGNORE = 'ignore'
PATH_DATA = 'data'

FILE_DATA = os.path.join(PATH_DATA, 'BASE DE DADOS PEDE 2024 - DATATHON.xlsx')
FILE_OLD_DATA = os.path.join(PATH_DATA, 'PEDE_PASSOS_DATASET_FIAP_ANTIGO.xlsx')
FILE_JOINED_DATA = os.path.join(PATH_DATA, 'data_joined.csv')

FILE_POSTGRES_PW = os.path.join(PATH_IGNORE, 'postgres-pw.txt')

In [3]:
dfs_year = {}

dfs_year[2022] = pd.read_excel(FILE_DATA, sheet_name=0)
dfs_year[2023] = pd.read_excel(FILE_DATA, sheet_name=1)
dfs_year[2024] = pd.read_excel(FILE_DATA, sheet_name=2)

# Dicionário

**INDE** - Índice de desenvolvimento educacional
- Ponderação entre os indicadores abaixo

**IAN** - Indicador de adequação de nível
- Registros administrativos
- Registra a defasagem da aprendizagem do aluno por meio da comparação da Fase atual do aluno da Associação, com a equivalência das Fases de ensido na Associação e a divisão dos anos escolares do ensino formal (por idade)
- Utiliza a fase atual, e a fase ideal conforme a idade.
- 10 (em fase), 5 (moderada) ou 2,5 (severa).

**IDA** - Indicador de desempenho acadêmico
- Notas Provas PM e Média geral Universitária
- Registra a proficiência do aluno nos exames padronizados de avaliação interna das disciplinas oferecidas pela Associação (os alunos das Fases 0 a 7). No caso dos alunos da Fase 8, esse indicador registra a nota média obtida pelos alunos em todas as disciplinas curriculares cursadas, nas respectivas instituições de ensino superior conveniadas ao programa de bolsas de estudo da Associação.
- Utiliza as notas internas da associação. Matemática + Português + Inglês / 3
- 0 a 10

**IEG** - Indicador de engajamento
- Registros de entrega de lição de casa e de voluntariado
- Mede o engajamento do aluno nas tarefas curriculares requeridas em cada uma das disciplinas oferecidas na Associação, e/ou seu engajamento em ações de voluntariado desenvolvidas pela Associação.
- Soma das pontuações das tarefas realizadas e registradas / Número de tarefas

**IAA** - Indicador de Autoavaliação
- Questionário de Autoavaliação individual
- Registra por meio de um questionário padronizado e adaptado às distintas faixas etárias dos alunoa a Associação, uma autoavaliação do aluno sobre como se sente consigo mesmo, sobre os estudos, sobre sua família, amigos e comunidade, e e sobre como se sente a respeito da Associação Passos Mágicos.
- Soma das pontuações das respostas do estudante / Número total de perguntas

**IPS** - Indicador Psicossocial
- Questionário individual de avaliação das psicólogas
- Avaliação da equipe de psicólogas para caracterizar o desenvolvimento do aluno nas suas interações familiares, no seu desenvolvimento emocional, comportamental e da sua socialização na vida comunitária. Esse indicador também caracteriza o tipo de atendimento psicológico oferecido pela Associação ao aluno.
- Avaliações feitas por **psicólogos**.
- Soma das pontuações / Número de avaliadores

**IPP** - Indicador Psicopedagógico
- Questionário individual de avaliação dos pedagogos e professores
- Avaliação da equipe de educadores e psicopedagogos para caracterizar o desenvolvimento cognitivo, emocional, comportamental e de socialização do aluno no seu processo de aprendizado dentro do Programa de Aceleração do Conhecimento, dos Programas Educacionais e das Atividades Culturais promovidas pela Associação.
- Soma das avaliações sobre aspectos **pedagógicos** / Número de avaliações

**IPV** - Indicador de Ponto de Virada
- Questionário Individual de avaliação dos pedagogos e professores
- Avaliação da equipe de educadores e psicopedagogos a respeito do desenvolvimento do aluno das aptidões necessárias para iniciar a transformação da sua vida por meio da Educação, avaliando a integração do aluno à Associação, o seu desenvolvimento emocional, e o seu potencial acadêmico.
- Análises longitudinais de progresso acadêmico, engajamento e desenvolvimento emocional

**Pedras** - Baseadas no INDE
- Quartzo: 3.0 < INDE < 6.1
- Ágata: 6.1 < INDE < 7.2
- Ametista: 7.2 < INDE < 8.2
- Topázio: 8.2 < INDE < 9.4

# Padronizando colunas
Algumas colunas estão nomeadas diferentemente entre anos. Vamos arrumar isso antes de proceder.

In [4]:
for year in dfs_year:
    print(dfs_year[year].columns)

Index(['RA', 'Fase', 'Turma', 'Nome', 'Ano nasc', 'Idade 22', 'Gênero',
       'Ano ingresso', 'Instituição de ensino', 'Pedra 20', 'Pedra 21',
       'Pedra 22', 'INDE 22', 'Cg', 'Cf', 'Ct', 'Nº Av', 'Avaliador1',
       'Rec Av1', 'Avaliador2', 'Rec Av2', 'Avaliador3', 'Rec Av3',
       'Avaliador4', 'Rec Av4', 'IAA', 'IEG', 'IPS', 'Rec Psicologia', 'IDA',
       'Matem', 'Portug', 'Inglês', 'Indicado', 'Atingiu PV', 'IPV', 'IAN',
       'Fase ideal', 'Defas', 'Destaque IEG', 'Destaque IDA', 'Destaque IPV'],
      dtype='object')
Index(['RA', 'Fase', 'INDE 2023', 'Pedra 2023', 'Turma', 'Nome Anonimizado',
       'Data de Nasc', 'Idade', 'Gênero', 'Ano ingresso',
       'Instituição de ensino', 'Pedra 20', 'Pedra 21', 'Pedra 22', 'Pedra 23',
       'INDE 22', 'INDE 23', 'Cg', 'Cf', 'Ct', 'Nº Av', 'Avaliador1',
       'Rec Av1', 'Avaliador2', 'Rec Av2', 'Avaliador3', 'Rec Av3',
       'Avaliador4', 'Rec Av4', 'IAA', 'IEG', 'IPS', 'IPP', 'Rec Psicologia',
       'IDA', 'Mat', 'Por', 'In

In [5]:
def check_column(
        df : pd.DataFrame,
        column : str
):
    ''' Checks if a column exists in a DataFrame '''
    if column in df.columns:
        return True
    else: 
        return False

In [6]:
def check_column_in_many(
        df_list : list,
        column : str,
        verbose : bool = False
):
    '''Checks if a column exists in all given DataFrames'''
    exists = True
    count = 0
    for df in df_list:
        column_exists = check_column(df, column)
        if column_exists != True:
            exists = False
            if verbose:
                print(f'Column does not exists in dataframe {count}')
        count += 1
    return exists

In [7]:
# Colunas que queremos no nossos dataframes padronizados
universal_columns = {
    'Nome Anonimizado':'nome_anonimizado',
    'Fase':'fase',
    # 'Idade':'idade',
    'Data de Nasc': 'ano_nascimento',
    'Gênero':'genero',
    'Ano ingresso':'ano_ingresso',
    'IAN':'ian',
    'IDA':'ida',
    'IEG':'ieg',
    'IAA':'iaa',
    'IPS':'ips',
    'IPP':'ipp',
    'IPV':'ipv',
    # 'Atingiu PV',
    'Mat':'matematica',
    'Por':'portugues',
    'Ing':'ingles',
    'Instituição de ensino':'instituicao'
}

In [8]:
uneven_columns = []

for column in universal_columns.keys():
    exists_in_all = check_column_in_many(
        df_list = list(dfs_year.values()),
        column = column
    )

    if exists_in_all == False:
        uneven_columns.append(column)

uneven_columns

['Nome Anonimizado', 'Data de Nasc', 'IPP', 'Mat', 'Por', 'Ing']

In [9]:
dfs_year[2022] = dfs_year[2022].rename(
    {'Nome' : 'Nome Anonimizado',
     'Ano nasc' : 'Data de Nasc',
     'Idade 22':'Idade',
     'Matem' : 'Mat',
     'Portug': 'Por',
     'Inglês' : 'Ing'},
     axis = 'columns',
     errors = 'raise'
)

In [10]:
uneven_columns = []

for column in universal_columns:
    # print(f'Checking {column}')
    exists_in_all = check_column_in_many(
        df_list = list(dfs_year.values()),
        column = column
    )

    if exists_in_all == False:
        uneven_columns.append(column)
    # print(exists_in_all)
    # print()
    
uneven_columns

['IPP']

In [11]:
# temporary
dfs_year[2022]['IPP'] = 0

## Mergendo dados

### Padronizando nomes das colunas

In [12]:
for year in dfs_year.keys():
    dfs_year[year] = dfs_year[year].rename(
        universal_columns,
        axis = 'columns',
        errors = 'raise'
    )
    dfs_year[year] = dfs_year[year][universal_columns.values()]

In [13]:
fixed_columns = ['nome_anonimizado', 'genero', 'ano_ingresso', 'ano_nascimento'] # will be taken from the oldest available

yearly_columns = list(universal_columns.values())
for col in fixed_columns:
    yearly_columns.remove(col)
yearly_columns # will be registered per-year

['fase',
 'ian',
 'ida',
 'ieg',
 'iaa',
 'ips',
 'ipp',
 'ipv',
 'matematica',
 'portugues',
 'ingles',
 'instituicao']

In [14]:
data = {}

for year in dfs_year.keys():
    data[year] = dfs_year[year][['nome_anonimizado'] + yearly_columns].copy()
    data[year] = data[year].set_index('nome_anonimizado')
    data[year].columns = [f'{col}_{year}' for col in data[year].columns if col != 'nome_anonimizado']

### Atualizando alunos de 2022 até 2024

In [15]:
df_2022 = dfs_year[2022][fixed_columns].copy()
df_2022 = df_2022.set_index('nome_anonimizado')

for year in dfs_year.keys():
    df_2022 = pd.merge(
        left = df_2022,
        right = data[year],
        on = 'nome_anonimizado',
        how = 'left'
    )
df_2022.tail(3)

,genero,ano_ingresso,ano_nascimento,fase_2022,ian_2022,ida_2022,ieg_2022,iaa_2022,ips_2022,ipp_2022,...,ida_2024,ieg_2024,iaa_2024,ips_2024,ipp_2024,ipv_2024,matematica_2024,portugues_2024,ingles_2024,instituicao_2024
nome_anonimizado,,,,,,,,,,,,,,,,,,,,,
Aluno-858,Menino,2022,2015,0,10.0,7.7,7.1,10.0,5.6,0,...,8.25,6.833333,9.002,2.51,7.500000,7.500000,10.0,6.5,NaN,Pública
Aluno-859,Menina,2022,2015,0,10.0,8.9,8.7,8.5,7.5,0,...,8.00,9.337121,8.002,7.51,8.958333,8.336667,8.0,8.0,NaN,Pública
Aluno-860,Menino,2022,2015,0,10.0,8.1,8.5,9.5,7.5,0,...,8.50,9.791667,9.002,7.51,8.750000,7.946667,8.0,9.0,NaN,Pública


In [16]:
df_2022.shape

(860, 39)

### Atualizando alunos de 2023 até 2024

In [17]:
df_2023 = dfs_year[2023][fixed_columns].copy()
df_2023 = df_2023.set_index('nome_anonimizado')

old_students = df_2022.index.to_list()
current_students = df_2023.index.to_list()

new_students = list(set(current_students) - set(old_students))

df_2023 = df_2023.loc[new_students]
df_2023.head(3)

,genero,ano_ingresso,ano_nascimento
nome_anonimizado,,,
Aluno-875,Masculino,2023,4/26/2014
Aluno-1042,Masculino,2023,2013-05-07 00:00:00
Aluno-1243,Feminino,2021,2003-04-12 00:00:00


In [18]:
df_2023['ano_nascimento'] = pd.to_datetime(df_2023['ano_nascimento']).dt.year
df_2023['ano_nascimento'] = df_2023['ano_nascimento'].astype(int)

In [19]:
for year in dfs_year.keys():
    df_2023 = pd.merge(
        left = df_2023,
        right = data[year],
        on = 'nome_anonimizado',
        how = 'left'
    )
df_2023.tail(3)

,genero,ano_ingresso,ano_nascimento,fase_2022,ian_2022,ida_2022,ieg_2022,iaa_2022,ips_2022,ipp_2022,...,ida_2024,ieg_2024,iaa_2024,ips_2024,ipp_2024,ipv_2024,matematica_2024,portugues_2024,ingles_2024,instituicao_2024
nome_anonimizado,,,,,,,,,,,,,,,,,,,,,
Aluno-1085,Feminino,2023,2011,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Aluno-1220,Feminino,2023,2005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,7.666667,8.555556,7.083,6.885,7.5,6.26,9.0,7.0,7.0,Concluiu o 3º EM
Aluno-1205,Feminino,2023,2005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
df_2023.shape

(414, 39)

### Pegando os alunos de 2024

In [21]:
df_2024 = dfs_year[2024][fixed_columns].copy()
df_2024 = df_2024.set_index('nome_anonimizado')

old_students = df_2022.index.to_list() + df_2023.index.to_list()
current_students = df_2024.index.to_list()

new_students = list(set(current_students) - set(old_students))

df_2024 = df_2024.loc[new_students]
df_2024.head(3)

,genero,ano_ingresso,ano_nascimento
nome_anonimizado,,,
Aluno-1430,Masculino,2024,2013-09-15
Aluno-1595,Masculino,2024,2010-02-23
Aluno-1339,Feminino,2024,2016-05-14


In [22]:
df_2024['ano_nascimento'] = pd.to_datetime(df_2024['ano_nascimento']).dt.year
df_2024['ano_nascimento'] = df_2024['ano_nascimento'].astype(int)

In [23]:
for year in dfs_year.keys():
    df_2024 = pd.merge(
        left = df_2024,
        right = data[year],
        on = 'nome_anonimizado',
        how = 'left'
    )
df_2024.tail(3)

,genero,ano_ingresso,ano_nascimento,fase_2022,ian_2022,ida_2022,ieg_2022,iaa_2022,ips_2022,ipp_2022,...,ida_2024,ieg_2024,iaa_2024,ips_2024,ipp_2024,ipv_2024,matematica_2024,portugues_2024,ingles_2024,instituicao_2024
nome_anonimizado,,,,,,,,,,,,,,,,,,,,,
Aluno-1377,Masculino,2024,2016,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.5,10.000,9.002,8.130,6.8750,6.9900,5.0,6.0,NaN,Pública
Aluno-1362,Masculino,2024,2017,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,8.5,6.875,10.002,5.635,6.8750,6.9150,8.0,9.0,NaN,Pública
Aluno-1476,Feminino,2024,2013,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,8.5,10.000,9.502,5.635,8.4375,8.7525,8.0,9.0,NaN,Pública


In [24]:
df_2024.shape

(387, 39)

### Juntando as sheets

In [25]:
df = pd.concat(
    [df_2022,
    df_2023]
)

df = pd.concat(
    [df,
     df_2024]
)

df.shape

(1661, 39)

## Tratando valores

In [26]:
df.dtypes

genero               object
ano_ingresso          int64
ano_nascimento        int64
fase_2022           float64
ian_2022            float64
ida_2022            float64
ieg_2022            float64
iaa_2022            float64
ips_2022            float64
ipp_2022            float64
ipv_2022            float64
matematica_2022     float64
portugues_2022      float64
ingles_2022         float64
instituicao_2022     object
fase_2023            object
ian_2023            float64
ida_2023            float64
ieg_2023            float64
iaa_2023            float64
ips_2023            float64
ipp_2023            float64
ipv_2023            float64
matematica_2023     float64
portugues_2023      float64
ingles_2023         float64
instituicao_2023     object
fase_2024            object
ian_2024            float64
ida_2024            float64
ieg_2024            float64
iaa_2024            float64
ips_2024            float64
ipp_2024            float64
ipv_2024            float64
matematica_2024     

### Strings

In [27]:
df.index

Index(['Aluno-1', 'Aluno-2', 'Aluno-3', 'Aluno-4', 'Aluno-5', 'Aluno-6',
       'Aluno-7', 'Aluno-8', 'Aluno-9', 'Aluno-10',
       ...
       'Aluno-1651', 'Aluno-1421', 'Aluno-1472', 'Aluno-1413', 'Aluno-1534',
       'Aluno-1584', 'Aluno-1366', 'Aluno-1377', 'Aluno-1362', 'Aluno-1476'],
      dtype='object', name='nome_anonimizado', length=1661)

In [28]:
df.index = df.index.str.capitalize()

In [29]:
cols_str = [
    'instituicao',
    'fase'
]

In [30]:
for col in cols_str:
    for year in dfs_year.keys():
        df[f'{col}_{year}'] = df[f'{col}_{year}'].astype(str)
        df[f'{col}_{year}'] = df[f'{col}_{year}'].str.lower()

### Floats

In [31]:
cols_float = [
    'ian',
    'ida',
    'ieg',
    'iaa',
    'ips',
    'ipp',
    'ipv',
    'matematica',
    'portugues',
    'ingles'
]

In [32]:
for col in cols_float:
    for year in dfs_year.keys():
        df[f'{col}_{year}'] = df[f'{col}_{year}'].astype(float)

### Ints

In [33]:
cols_int = [
    'ano_nascimento',
]

In [34]:
for col in cols_int:
    df[f'{col}'] = df[f'{col}'].astype(int)

In [35]:
df.dtypes

genero               object
ano_ingresso          int64
ano_nascimento        int64
fase_2022            object
ian_2022            float64
ida_2022            float64
ieg_2022            float64
iaa_2022            float64
ips_2022            float64
ipp_2022            float64
ipv_2022            float64
matematica_2022     float64
portugues_2022      float64
ingles_2022         float64
instituicao_2022     object
fase_2023            object
ian_2023            float64
ida_2023            float64
ieg_2023            float64
iaa_2023            float64
ips_2023            float64
ipp_2023            float64
ipv_2023            float64
matematica_2023     float64
portugues_2023      float64
ingles_2023         float64
instituicao_2023     object
fase_2024            object
ian_2024            float64
ida_2024            float64
ieg_2024            float64
iaa_2024            float64
ips_2024            float64
ipp_2024            float64
ipv_2024            float64
matematica_2024     

## Exportando base nova

In [36]:
df.to_csv(FILE_JOINED_DATA)

# Tratando dados (TBA)
Adicionando novos parâmetros, e lidando com dados nulos

# Conexão com o PostgreSQL

In [37]:
def connect_to_db(
        dbname = 'datathon',
        user = 'postgres',
        password_path = FILE_POSTGRES_PW,
        host = 'localhost',
        port = '5432'
):
    password = ''
    with open(password_path) as f:
        password = f.read()

    conn = ps.connect(
        dbname = dbname,
        user = user,
        password = password,
        host = host,
        port = port
    )

    return conn

In [38]:
cursor = None
conn = None
try:
    conn = connect_to_db()
    sql_create_schema = 'CREATE SCHEMA IF NOT EXISTS datathon;'
    
    cursor = conn.cursor()
    cursor.execute(sql_create_schema)
    conn.commit()

    print("Schema created successfully")
except Exception as e:
    
    print(f'Failed to create schema | {e}')
    if conn is not None:
        conn.rollback()
finally:
    if cursor is not None:
        cursor.close()
    if conn is not None:
        conn.close()

Schema created successfully


In [46]:
sql_types = {
    'object' : 'VARCHAR(255)',
    'int64': 'INT',
    'float64': 'FLOAT'
}

schema_types = {}

for col in df.columns:
    schema_types[col] = sql_types[f'{df[col].dtype}']

schema_types

{'genero': 'VARCHAR(255)',
 'ano_ingresso': 'INT',
 'ano_nascimento': 'INT',
 'fase_2022': 'VARCHAR(255)',
 'ian_2022': 'FLOAT',
 'ida_2022': 'FLOAT',
 'ieg_2022': 'FLOAT',
 'iaa_2022': 'FLOAT',
 'ips_2022': 'FLOAT',
 'ipp_2022': 'FLOAT',
 'ipv_2022': 'FLOAT',
 'matematica_2022': 'FLOAT',
 'portugues_2022': 'FLOAT',
 'ingles_2022': 'FLOAT',
 'instituicao_2022': 'VARCHAR(255)',
 'fase_2023': 'VARCHAR(255)',
 'ian_2023': 'FLOAT',
 'ida_2023': 'FLOAT',
 'ieg_2023': 'FLOAT',
 'iaa_2023': 'FLOAT',
 'ips_2023': 'FLOAT',
 'ipp_2023': 'FLOAT',
 'ipv_2023': 'FLOAT',
 'matematica_2023': 'FLOAT',
 'portugues_2023': 'FLOAT',
 'ingles_2023': 'FLOAT',
 'instituicao_2023': 'VARCHAR(255)',
 'fase_2024': 'VARCHAR(255)',
 'ian_2024': 'FLOAT',
 'ida_2024': 'FLOAT',
 'ieg_2024': 'FLOAT',
 'iaa_2024': 'FLOAT',
 'ips_2024': 'FLOAT',
 'ipp_2024': 'FLOAT',
 'ipv_2024': 'FLOAT',
 'matematica_2024': 'FLOAT',
 'portugues_2024': 'FLOAT',
 'ingles_2024': 'FLOAT',
 'instituicao_2024': 'VARCHAR(255)'}

In [ ]:
sql_columns = ''
i = 0

for col in schema_types:
    col_type = schema_types[col]
    if i > 0:
        sql_columns += ', '
    sql_columns += f'{col} {col_type}'
    i += 1

sql_columns

'genero VARCHAR(255), ano_ingresso INT, ano_nascimento INT, fase_2022 VARCHAR(255), ian_2022 FLOAT, ida_2022 FLOAT, ieg_2022 FLOAT, iaa_2022 FLOAT, ips_2022 FLOAT, ipp_2022 FLOAT, ipv_2022 FLOAT, matematica_2022 FLOAT, portugues_2022 FLOAT, ingles_2022 FLOAT, instituicao_2022 VARCHAR(255), fase_2023 VARCHAR(255), ian_2023 FLOAT, ida_2023 FLOAT, ieg_2023 FLOAT, iaa_2023 FLOAT, ips_2023 FLOAT, ipp_2023 FLOAT, ipv_2023 FLOAT, matematica_2023 FLOAT, portugues_2023 FLOAT, ingles_2023 FLOAT, instituicao_2023 VARCHAR(255), fase_2024 VARCHAR(255), ian_2024 FLOAT, ida_2024 FLOAT, ieg_2024 FLOAT, iaa_2024 FLOAT, ips_2024 FLOAT, ipp_2024 FLOAT, ipv_2024 FLOAT, matematica_2024 FLOAT, portugues_2024 FLOAT, ingles_2024 FLOAT, instituicao_2024 VARCHAR(255)'

In [55]:
sql_create_table = f'DROP TABLE IF EXISTS datathon.passos_magicos; CREATE TABLE IF NOT EXISTS datathon.passos_magicos({sql_columns})'
sql_create_table

'DROP TABLE IF EXISTS datathon.passos_magicos; CREATE TABLE IF NOT EXISTS datathon.passos_magicos(genero VARCHAR(255), ano_ingresso INT, ano_nascimento INT, fase_2022 VARCHAR(255), ian_2022 FLOAT, ida_2022 FLOAT, ieg_2022 FLOAT, iaa_2022 FLOAT, ips_2022 FLOAT, ipp_2022 FLOAT, ipv_2022 FLOAT, matematica_2022 FLOAT, portugues_2022 FLOAT, ingles_2022 FLOAT, instituicao_2022 VARCHAR(255), fase_2023 VARCHAR(255), ian_2023 FLOAT, ida_2023 FLOAT, ieg_2023 FLOAT, iaa_2023 FLOAT, ips_2023 FLOAT, ipp_2023 FLOAT, ipv_2023 FLOAT, matematica_2023 FLOAT, portugues_2023 FLOAT, ingles_2023 FLOAT, instituicao_2023 VARCHAR(255), fase_2024 VARCHAR(255), ian_2024 FLOAT, ida_2024 FLOAT, ieg_2024 FLOAT, iaa_2024 FLOAT, ips_2024 FLOAT, ipp_2024 FLOAT, ipv_2024 FLOAT, matematica_2024 FLOAT, portugues_2024 FLOAT, ingles_2024 FLOAT, instituicao_2024 VARCHAR(255))'

In [56]:
try:
    conn = connect_to_db()
    cursor = conn.cursor()

    # 4. Execute the CREATE TABLE statement
    cursor.execute(sql_create_table)

    # 5. Commit the transaction
    conn.commit()
    print("Table created successfully!")
except Exception as e:
    print(f'Failed to create schema | {e}')
    if conn is not None:
        conn.rollback()
finally:
    # 6. Close the cursor and connection
    if cursor is not None:
        cursor.close()
    if conn is not None:
        conn.close()

Table created successfully!


## Subindo dados

In [57]:
def ingest_data(file, table = 'datathon.passos_magicos'):
    conn = connect_to_db()
    cursor = conn.cursor()
    try:
        with open(file, 'r') as file:
            next(file) # skips header
            cursor.copy_expert(f'COPY {table} FROM STDIN WITH (FORMAT CSV)', file)
    except Exception as e:
        conn.rollback()
        print(f'Failed to insert | {e}')
    finally:
        conn.commit()
        if cursor is not None:
            cursor.close()
        if conn is not None:
            conn.close()

---

# Perguntas norteadoras

## 1. Adequação do nível (IAN): Qual é o perfil geral de defasagem dos alunos (IAN) e como ele evolui ao longo do ano?
(Ex.: quantos alunos estão moderadamente ou severamente defasados?)

## 2. Desempenho acadêmico (IDA): O desempenho acadêmico médio (IDA) está melhorando, estagnado ou caindo ao longo das fases e anos?

## 3. Engajamento nas atividades (IEG): O grau de engajamento dos alunos (IEG) tem relação direta com seus indicadores de desempenho (IDA) e do ponto de virada (IPV)?

## 4. Autoavaliação (IAA): As percepções dos alunos sobre si mesmos (IAA) são coerentes com seu desempenho real (IDA) e engajamento (IEG)?

## 5. Aspectos psicossociais (IPS): Há padrões psicossociais (IPS) que antecedem quedas de desempenho acadêmico ou de engajamento?

## 6. Aspectos psicopedagógicos (IPP): As avaliações psicopedagógicas (IPP) confirmam ou contradizem a defasagem identificada pelo IAN?

## 7. Ponto de virada (IPV): Quais comportamentos - acadêmicos, emocionais ou de engajamento - mais influenciam o IPV ao longo do tempo?

## 8. Multidimensionalidade dos indicadores: Quais combinações de indicadores (IDA + IEG + IPS + IPP) melhor explicam o desempenho global do aluno (INDE)?

## 9. Previsão de risco com Machine Learning: Quais padrões nos indicadores permitem identificar alunos em risco antes de queda no desempenho ou aumento da defasagem? Construa um modelo preditivo que mostre uma probabilidade do aluno ou aluna entrar em risco de defasagem.

## 10. Efetividade do programa: Os indicadores mostram melhora consistente ao longo do ciclo nas diferentes fases (Quartzo, Ágata, Ametista e Topázio), confirmando o impacto real do programa?

## 11. Insights e criatividade: Você pode adicionar mais insights e pontos de vista não abordados nas perguntas, utilize a criatividade e a análise dos dados para trazer sugestões para a Passos Mágicos.

## Quais padrões nos indicadores permitem identificar alunos em risco antes de queda no desempenho ou aumento da defasagem?